<a href="https://colab.research.google.com/github/chriskary1/retail_store_inventory/blob/main/retail_store_inventory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from prophet import Prophet


!pip install gdown
import gdown

#
file_id = '1d_D6eXCFZqElrTw-g74w_bjyAcjUb9bG'
url = f'https://drive.google.com/uc?id={file_id}'

output = 'retail_store_inventory.csv'
gdown.download(url, output, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1d_D6eXCFZqElrTw-g74w_bjyAcjUb9bG
To: /content/retail_store_inventory.csv
100%|██████████| 6.19M/6.19M [00:00<00:00, 145MB/s]


'retail_store_inventory.csv'

In [4]:
# read file as dataset
df_a = pd.read_csv('retail_store_inventory.csv')
df_a.head()

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn
1,2022-01-01,S001,P0002,Toys,South,204,150,66,144.04,63.01,20,Sunny,0,66.16,Autumn
2,2022-01-01,S001,P0003,Toys,West,102,65,51,74.02,27.99,10,Sunny,1,31.32,Summer
3,2022-01-01,S001,P0004,Toys,North,469,61,164,62.18,32.72,10,Cloudy,1,34.74,Autumn
4,2022-01-01,S001,P0005,Electronics,East,166,14,135,9.26,73.64,0,Sunny,0,68.95,Summer


In [7]:

# select Date, Units Sold
df = df_a[['Date', 'Units Sold']].copy()

# rename columns
df.rename(columns={'Date': 'ds', 'Units Sold': 'y'}, inplace=True)

# check
print(df.head())
print(df.info())


           ds    y
0  2022-01-01  127
1  2022-01-01  150
2  2022-01-01   65
3  2022-01-01   61
4  2022-01-01   14
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73100 entries, 0 to 73099
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ds      73100 non-null  object
 1   y       73100 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ MB
None


In [8]:
df['ds'] = pd.to_datetime(df['ds'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73100 entries, 0 to 73099
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      73100 non-null  datetime64[ns]
 1   y       73100 non-null  int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 1.1 MB


In [9]:

#create model m
m = Prophet()
m.fit(df)



INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpksaphz33/tqpf5fho.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpksaphz33/netr70zn.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=19801', 'data', 'file=/tmp/tmpksaphz33/tqpf5fho.json', 'init=/tmp/tmpksaphz33/netr70zn.json', 'output', 'file=/tmp/tmpksaphz33/prophet_modelj3qsdz4q/prophet_model-20250720163650.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:36:50 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:37:08 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [10]:
future = m.make_future_dataframe(periods=365, freq='D')
future.tail()

forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
1091,2024-12-27,135.037875,-1.421839,270.639806
1092,2024-12-28,133.635141,-7.868151,265.436408
1093,2024-12-29,135.264466,-3.403226,272.525391
1094,2024-12-30,133.688232,-8.081348,274.244183
1095,2024-12-31,136.025539,-10.393710,271.541369


In [11]:
from prophet.plot import plot_plotly, plot_components_plotly
plot_plotly(m, forecast)



In [12]:
plot_components_plotly(m, forecast)

/usr/local/lib/python3.11/dist-packages/plotly/io/_json.py:560: UserWarning:

Discarding nonzero nanoseconds in conversion.



In [ ]:
# Εξαγωγή πλήρους forecast σε CSV
forecast.to_csv('forecast_results.csv', index=False)

# Λήψη του αρχείου τοπικά (Google Colab)
from google.colab import files
files.download('forecast_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>